## Optimizing Resonator Designs

There are many factors for the resonator design. I categorized the factors into two tiers:

Top Tier:
1. Length
2. Center pin to gap ratio (this defines the impedance of the resonator). 

Second Tier:
1. Center pin width


In [21]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

In [41]:
# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 250*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width

### Use Qiskit-metal function to calculate transmission line pin/gap ratio

Note to self: when constructing function for scipy, the variable to optimize has to be dimensionless. 

In [42]:
import qiskit_metal.analyses as analyses

In [43]:
def f(line_gap):
    return(analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)[4]-50)

In [44]:
freq = 7*u.GHz.si
line_width = 15.322*u.um
line_gap = 10*u.um

In [27]:
res = analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap.to(u.m).value,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)

Z0 = res[4]*u.Ohm

In [28]:
Z0

<Quantity 52.72897025 Ohm>

In [56]:
optimized_gap = optimize.fsolve(f,line_gap.si.value)[0]*u.m

In [57]:
optimzied_gap

<Quantity 8.28349609 um>

In [58]:
f(optimized_gap.si.value)

7.105427357601002e-15

In [60]:
analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     optimized_gap.to(u.m).value,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)

(1.6655452372229007e-09,
 4.211256177953425e-07,
 1.6845024711813694e-10,
 3.703063394264467e-05,
 50.00000000000001,
 6.082601695790244,
 9.583285084677287e-11)

### Using the optimzied pin/gap ratio optimzie to construct a python function that takes coupling strength between and outputs a coupled line Tee design. 